# BNNs

In [1]:
from bayesian_models import Pbnn
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from doepy import build
import pickle
from scipy.stats import norm, uniform, lognorm
from scipy.stats import qmc    #for sobol seq. (LHS is also available in this QuasiMC library)

In [2]:
from LStates.g11D_electric import example_electric

function = example_electric

def convert_lognormal(mean_ln, std_ln):
    gaussian_param = np.zeros(2)

    SigmaLogNormal = np.sqrt( np.log(1+(std_ln/mean_ln)**2))
    MeanLogNormal = np.log( mean_ln ) - SigmaLogNormal**2/2

    gaussian_param[0] = MeanLogNormal
    gaussian_param[1] = SigmaLogNormal

    return gaussian_param

In [3]:
# X1 = L (m) mean=4.2    cov=0.10   lognormal/R>0
x1_mean = 4.20
x1_std = x1_mean * 0.10
normal_std_1 = np.sqrt(np.log(1 + (x1_std/x1_mean)**2))
normal_mean_1 = np.log(x1_mean) - normal_std_1**2 / 2

# X2 = h (m) mean=0.02    cov=0.10 lognormal /R>0
x2_mean = 0.02
x2_std = x2_mean * 0.1
normal_std_2 = np.sqrt(np.log(1 + (x2_std/x2_mean)**2))
normal_mean_2 = np.log(x2_mean) - normal_std_2**2 / 2

# X3 = d (m) mean=0.001  cov=0.05 lognormal /R>0
x3_mean = 0.001
x3_std = x3_mean * 0.05
normal_std_3 = np.sqrt(np.log(1 + (x3_std/x3_mean)**2))
normal_mean_3 = np.log(x3_mean) - normal_std_3**2 / 2

# X4 = ZL () mean=1000    cov=0.20 lognormal /R>0
x4_mean = 1000
x4_std = x4_mean * 0.2
normal_std_4 = np.sqrt(np.log(1 + (x4_std/x4_mean)**2))
normal_mean_4 = np.log(x4_mean) - normal_std_4**2 / 2

# X5 = Z0 () mean=50   cov=0.05 lognormal /R>0
x5_mean = 50
x5_std = x5_mean * 0.05
normal_std_5 = np.sqrt(np.log(1 + (x5_std/x5_mean)**2))
normal_mean_5 = np.log(x5_mean) - normal_std_5**2 / 2

# X6 = ae (V/m) mean=1  cov=0.20 lognormal /R>0
x6_mean = 1
x6_std = x6_mean * 0.2
normal_std_6 = np.sqrt(np.log(1 + (x6_std/x6_mean)**2))
normal_mean_6 = np.log(x6_mean) - normal_std_6**2 / 2

# X7 = theta_e (rad) mean=pi/4    cov=0.577 uniform / [0,pi/2]
x7_min = 0
x7_max = np.pi / 2

# X8 = theta_p (rad) mean=pi/4    cov=0.577 uniform / [0,pi/2]
x8_min = 0
x8_max = np.pi / 2

# X9 = phi_p (rad) mean=pi   cov=0.577 uniform / [0,pi*2]
x9_min = 0
x9_max = np.pi*2

# X10 = f (MHz) mean=30    cov=0.096 uniform / [25 ,35]
x10_min = 25e6
x10_max = 35e6

# X11 = alpha (-) mean=0.0010  cov=0.289 uniform / [0.0005 , 0.0015]
x11_min = 0.0005
x11_max =  0.0015

In [4]:
dim = 11
n_mcs = int(1e6)
X = np.zeros((n_mcs,dim))
L = X[:,0] = np.random.lognormal(mean=normal_mean_1, sigma=normal_std_1, size=n_mcs)   #L (m)
h = X[:,1] = np.random.lognormal(mean=normal_mean_2, sigma=normal_std_2, size=n_mcs)   #h (m)
d = X[:,2] = np.random.lognormal(mean=normal_mean_3, sigma=normal_std_3, size=n_mcs)   #d (m)
ZL = X[:,3] = np.random.lognormal(mean=normal_mean_4, sigma=normal_std_4, size=n_mcs)   #ZL ()
Z0 = X[:,4] = np.random.lognormal(mean=normal_mean_5, sigma=normal_std_5, size=n_mcs)   #Z0 ()
ae = X[:,5] = np.random.lognormal(mean=normal_mean_6, sigma=normal_std_6, size=n_mcs)   #ae (V/m)
theta_e = X[:,6] = np.random.uniform(low=x7_min, high=x7_max, size=n_mcs)        #theta_e (rad)
theta_p = X[:,7] = np.random.uniform(low=x8_min, high=x8_max, size=n_mcs)        #theta_p (rad)
phi_p =  X[:,8] = np.random.uniform(low=x9_min, high=x9_max, size=n_mcs)        #phi_p (rad)
f = X[:,9] = np.random.uniform(low=x10_min, high=x10_max, size=n_mcs)      #f (MHz)
alpha = X[:,10] = np.random.uniform(low=x11_min, high=x11_max, size=n_mcs)     #alpha (-)

y_test = function(X)
y_max = np.max(y_test)
Pf_ref = np.sum( y_test < 0 ) / n_mcs
B_ref= - norm.ppf( Pf_ref )
Pf_ref, B_ref

(0.000218, 3.517276105428664)

In [5]:
exp = {'x1': [x1_mean, x1_std, 'lognormal'],
 'x2': [x2_mean, x2_std, 'lognormal'],
 'x3': [x3_mean, x3_std, 'lognormal'],
 'x4': [x4_mean, x4_std, 'lognormal'],
 'x5': [x5_mean, x5_std, 'lognormal'],
 'x6': [x6_mean, x6_std, 'lognormal'],
 'x7': [x7_min, x7_max, 'uniform'],
 'x8': [x8_min, x8_max, 'uniform'],
 'x9': [x9_min, x9_max, 'uniform'],
 'x10': [x10_min, x10_max, 'uniform'],
 'x11': [x11_min, x11_max, 'uniform']}

In [7]:
number_experiments = 1
passive_samples = 50
active_samples = 50
output = 1
layers, archit = 3 , [30, 30, 30]  #change archit size if layers are increased
bnn_simulations = 100
training_epochs = 5000
n_mcs = int(1e6)
learning_points = 10

for experiments in range(number_experiments):
    print('Experiment: ', experiments+1 , '########################################################' )
    
    ActiveTrain_1 = {}

    #PASSIVE TRAINING---------------------------------------------------
    X = np.zeros((passive_samples, dim))
    X_norm = np.zeros((passive_samples, dim))
   
    exp_norm = {}
    for var_name in range(dim):
        exp_norm['x'+ str(var_name+1)] = [0.000001, 0.999999]    #initial design domain for each variable (normal, uniform)

    #Latin hypercube sampling
    Xdoe = build.space_filling_lhs(exp_norm , num_samples = passive_samples)

    for margin in range (0, dim):
        var = 'x' + str (margin + 1)
        X_norm[:, margin] = Xdoe[var]

        if exp[var][2] == 'normal':
            loc_ = exp[var][0]
            scale_ = exp[var][1]
            X[:, margin] = norm.ppf(Xdoe[var], loc=loc_, scale=scale_)

        elif exp[var][2] == 'uniform':
            loc_ = exp[var][0]
            scale_ = exp[var][1]
            X[:, margin] = uniform.ppf(Xdoe[var], loc=loc_, scale=scale_-loc_)

        elif exp[var][2] == 'lognormal':
            xlog_mean = exp[var][0]
            xlog_std = exp[var][1]
            gaussian_param = convert_lognormal(xlog_mean, xlog_std)
            X[:, margin] = lognorm.ppf(Xdoe[var], s=gaussian_param[1], scale=xlog_mean) 

    Y = function(X)

    #ACTIVE TRAIN LOOP ---------------------------------------------------
    for active in range(passive_samples, active_samples+passive_samples+1):
        #-------------------------------------------creating bnn
        #setting up the network architecture -----------------------------------
        config = {"n_infeatures": dim,
                "n_outfeatures": output,
                "n_samples": len(X_norm),
                "learn_all_params": False,  #to learn mean and sigma
                "fixed_param": 0.001} 
        
        ModelName = 'BNN_' + str(len(X_norm))
        mybnn = Pbnn(config)
        
        mybnn.build_bnn(layers, archit) #----------------------------------------------------------MODEL ARCHITECTURE
        #-------------------------------------------training bnn
        
        # batch_size = len(X)
        batch_size = 8
        # batch_size = [np.floor_divide(len(X), 10)][0] + 1  # splitting DoE

        train_env = {"batch_size": batch_size,
                    "learning_rate": 0.001,
                    "epochs": training_epochs,
                    "callback_patience": 500,
                    "verbose": 0,
                    "valid_split":0.0}
        
        print('Training BNN With', len(X_norm), 'samples' )
        history = mybnn.train_bnn(X_norm, Y, train_env)

        #-------------------------------------------MC population
        Xtest = np.zeros((int(n_mcs), dim))
        MCinputs_norm= np.random.uniform(0.000001, 0.999999, size=(int(n_mcs), dim))

        for margin in range (0, dim):
            var = 'x' + str (margin + 1)

            if exp[var][2] == 'normal':
                loc_ = exp[var][0]
                scale_ = exp[var][1]
                Xtest[:, margin] = norm.ppf(MCinputs_norm[:, margin], loc=loc_, scale=scale_)

            elif exp[var][2] == 'uniform':
                loc_ = exp[var][0]
                scale_ = exp[var][1]
                Xtest[:, margin] = uniform.ppf(MCinputs_norm[:, margin], loc=loc_, scale=scale_-loc_)

            elif exp[var][2] == 'lognormal':
                xlog_mean = exp[var][0]
                xlog_std = exp[var][1]
                gaussian_param = convert_lognormal(xlog_mean, xlog_std)
                Xtest[:, margin] = lognorm.ppf(MCinputs_norm[:, margin], s=gaussian_param[1], scale=xlog_mean) 

        #-------------------------------------------model predictions over MC population
        print('BNN predictions with MC population...')
        Mean_muY_MC, Stdv_muY_MC, Mean_sigmaY_MC, Stdv_sigmaY_MC = mybnn.modeluq_bnn(MCinputs_norm, nsim = bnn_simulations)
        y_mcs = function(Xtest)

        PF = np.sum(Mean_muY_MC < 0) / n_mcs
        B = - norm.ppf( PF )
        PF_ref = np.sum(y_mcs < 0) / n_mcs

        print('PF_ref =', PF_ref, 'PF =', PF, 'and B =',"%.5f" % round(B, 3) ,'-------------------------------')
        print(' ')
        #-------------------------------------------Selecting new training point
        U_f = np.abs(Mean_muY_MC) / Stdv_muY_MC
        U_min_args = np.argsort(U_f.reshape(-1))     #ordering arguments from min to max
        X_new = Xtest[U_min_args[:learning_points]]  #choosing a given number of MC samples from the minimum U values
        X_new_norm = MCinputs_norm[U_min_args[:learning_points]]  #choosing a given number of MC samples from the minimum U values
        X_norm = np.concatenate((X_norm, X_new_norm), axis=0)

        Y_new = function(X_new)
        Y = np.concatenate((Y, Y_new), axis=0)

        #-------------------------------------------Saving results
        ActiveTrain_1[ModelName] = mybnn.weights, PF, B
        filename1 = 'Batch_'+ str(experiments+1)+'.sav'
    
        pickle.dump(ActiveTrain_1, open(filename1, 'wb'))

Experiment:  1 ########################################################
Training BNN With 50 samples
Training completed
Minimum loss:  13.645233154296875
BNN predictions with MC population...
PF_ref = 0.000242 PF = 0.012464 and B = 2.24300 -------------------------------
 
Training BNN With 60 samples
Training completed
Minimum loss:  10.13916301727295
BNN predictions with MC population...
PF_ref = 0.000234 PF = 0.035724 and B = 1.80300 -------------------------------
 
Training BNN With 70 samples
Training completed
Minimum loss:  8.864086151123047
BNN predictions with MC population...
PF_ref = 0.000247 PF = 0.020285 and B = 2.04800 -------------------------------
 
Training BNN With 80 samples
Training completed
Minimum loss:  5.568042278289795
BNN predictions with MC population...
PF_ref = 0.000239 PF = 0.022759 and B = 2.00000 -------------------------------
 
Training BNN With 90 samples
Training completed
Minimum loss:  4.987165451049805
BNN predictions with MC population...
PF_r

to sample with sobol

In [ ]:
sampler = qmc.Sobol(d=2, scramble=True)    #d=dimensionality
sample = sampler.random_base2(m=5)   #change m=exponent to increase the sample size

l_bounds = [-2.0, -2.0]  #design domain for each variable in the physical space
u_bounds = [2.0, 2.0]
X = qmc.scale(sample, l_bounds, u_bounds)

Y = function(X)
passive_samples = len(X)
print(passive_samples)

Select MULTIPLE NEW training samples

In [ ]:
learning_points = 5
#-------------------------------------------Selecting MULTIPLE NEW training point
# U_f = np.abs(Mean_muY_MC) / Stdv_muY_MC
U_f = Stdv_muY_MC

U_min_args = np.argsort(U_f.reshape(-1))     #ordering arguments from min to max
X_new = Xtest[U_min_args[:learning_points]]  #choosing a given number of MC samples from the minimum U values
X = np.concatenate((X, X_new), axis=0)

Y_new = function(X_new)
Y = np.concatenate((Y, Y_new), axis=0)
len(Y)

Select SINGLE NEW Training point

In [ ]:
#-------------------------------------------Selecting SINGLE NEW training point
U_f = np.abs(Mean_muY_MC) / Stdv_muY_MC
U_min = np.argmin(U_f)
X_new = Xtest[U_min].reshape(-1, dim)
# X = np.concatenate((X, X_new), axis=0)

Y_new = function(X_new)
# Y = np.concatenate((Y, Y_new), axis=0)
print(Y)